In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import math
import imageio
import glob2 as glob
import os

from skimage.measure import compare_ssim as ssim

In [4]:
path = "/home/fabiana/Desktop/projeto-final-src/datasets/HandGesture/"

In [5]:
def getAllDirectories (pathToDirectory):
    '''
        Function that gets all the subdirectories of a folder
        
        Args:
            path: Path to root directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    files = glob.glob(pathToDirectory + '*/', recursive=True)
    return files

def getAllFiles (pathToFiles):
    '''
        Function that gets all the filenames from a directory
        
        Args:
            path: Path to directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    files = glob.glob(pathToFiles + '*/video_*/', recursive=True)
    return files

In [6]:
directoriesList = getAllDirectories(path)
filesList = getAllFiles(path)

In [228]:
# def extractFramesToFile (pathToFile, dest):

#     vidObj = cv2.VideoCapture(pathToFile)

#     NUMBER_OF_FRAMES = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    
#     print(NUMBER_OF_FRAMES)

#     #Capture first frame
#     read0, frame0 = vidObj.read()
    
#     if (not read0):
#         print ("Error while reading first frame")
#         return

#     init_gray = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
#     cv2.imwrite(dest + str(0) + ".jpg", init_gray)

#     for j in range(0, NUMBER_OF_FRAMES):
#         # Capture second frame.
#         success, frame1 = vidObj.read()
        
#         if success:
#             # Convert frames to grayscale. (ie. previous and current frames)
#             gray0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
#             gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

#             # Display the current frame.
# #             cv.imshow('frame1',gray1)
#             cv2.imwrite(dest + str(j) + ".jpg", gray1)
        
#         else:
# #             print ("Error while reading " + str(j) + "th frame")
#             continue
            
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#         # Set previous frame.
#         ret0, frame0 = vidObj.read()
#         if (not ret0):
#             break

#     # When everything done, release the capture

#     vidObj.release()
#     cv2.destroyAllWindows()

In [223]:
#It was used to get frames from video and save them as images

# for path in directoriesList:
#     innerFiles = glob.glob(path + '/*')
#     for j, innerPath in enumerate(innerFiles):
#         writePath = os.path.join(path, "video_" + str(j) + "/")
#         os.mkdir(writePath)
#         extractFramesToFile(innerPath, writePath)

In [10]:
#Initializing variables

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 10

NUMBER_OF_NP_CANDIDATES = 10

# Scale factor. (prev .2) (pretty good 0.6)
F = 0.9

# Cr probability value. (prev .5) (pretty good 0.9)
Cr = 0.6

In [11]:
# Calculate ASSIM for a chromosome.
def getASSIM(KF, filepath):
    ssim_sum = 0
    for i in range(0, TOTAL_KEY_FRAMES - 2):
        while True:
            try:
                im1 = cv2.imread(filepath + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(filepath + str(KF[i+1]) + ".jpg",0)
                ssim_sum += ssim(im1, im2)
            except:
#                 print i, KF, KF[i], KF[i+1]
#                 continue
                print (i, KF[i], KF[i+1])
                raise
            break
    return ssim_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and ASSIMs).
def initialize_NP(maxNumberOfFrames, filepath):
    print (maxNumberOfFrames)
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(0, maxNumberOfFrames), TOTAL_KEY_FRAMES)))
        NP[-1].append(getASSIM(NP[-1], filepath))
#         print NP[-1]


# MUTATION
def mutation(parent, maxNumberOfFrames, filepath):
    R = random.sample(NP,2)
    global MV
    MV[:] = []
    MV_value = 0
#     print NP[parent]
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[parent][i] + F*(R[0][i] - R[1][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > maxNumberOfFrames-1):
            MV.append(maxNumberOfFrames-1)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getASSIM(MV, filepath))

    
# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant, filepath):
    print ("mutant: ", mutant)
    print ("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getASSIM(TV, filepath))

# SELECTION : Selects offspring / parent based on lower ASSIM value.
def selection(parent, trail_vector):
    if(trail_vector[-1] < parent[-1]):
        parent[:] = trail_vector
        print ("yes", parent)
    else:
        print ("no")

# bestParent returns the parent with then minimum ASSIM value.
def bestParent(population):
    Min_ASSIM_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] < Min_ASSIM_value):
            Min_ASSIM_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index

In [ ]:
for i, framesPath in enumerate(filesList):
    # Population matrix.
    NP = []
    # Mutation vector.
    MV = []
    # Trail vector.
    TV = []
    list = os.listdir(framesPath) # dir is your directory path
    framesQnt = len(list)
    
    print ("Starting routine for file " + framesPath)
    print ("Quantity of frames: " + str(framesQnt))
    
    initialize_NP(framesQnt, framesPath)
    
    for GENERATION in range(STOPPING_ITERATION):
        for i in range(NUMBER_OF_NP_CANDIDATES):
            print ("---------------------", "PARENT:", i+1 , "GENERATION:", GENERATION+1, "---------------------")
            mutation(i, framesQnt, framesPath)
            crossover(NP[i], MV, framesPath)
            selection(NP[i], TV)
            TV[:] = []
    best_parent = bestParent(NP)
    print ("best solution is: ", best_parent)
    
    images_for_gif = []
    for frame_number in best_parent[:-1]:
        images_for_gif.append(imageio.imread(framesPath + str(frame_number) + ".jpg"))
    imageio.mimsave(framesPath + "result.gif", images_for_gif)

In [1]:
def buildImageHistogram(descriptor_list):
    print("Generating histogram for image")
    histogram = np.zeros(len(self.kmeans_ret.cluster_centers_))
    cluster_result = self.kmeans_ret.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram